# Image Processing


## Learning objectives

* Understand how scalar data and color images are stored as multi-dimensional arrays
* Access multi-dimensional data using multiple indices
* Locate discrete objects using image segmentation; quantify their location, size, and shape
* Load and use the external libraries `scikit-image` and `matplotlib` and use their documentation

## Downloading images from GitHub Directly

In [ ]:
import requests
from io import BytesIO
from PIL import Image

In [ ]:
url = 'https://raw.githubusercontent.com/lblogan14/CDSA2022_CompCourse/main/Day_10/300K-Nuclei-C8BTBT_crop.png'
req = requests.get(url)
Image.open(BytesIO(req.content))

## Exercise 1: What is an image?

You can think of a black & white picture as a 2D array of values, where each element of the array contains the brightness at that location.  

For example:

In [ ]:
import skimage                  # for image-processing
from skimage import io          # give ourselves a shortcut 
from skimage import measure    # give ourselves a shortcut 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

mysteryimage = np.zeros((8, 12))
mysteryimage[::2, 1::2] = 80      # make a plaid pattern
mysteryimage[1::2, ::2] = 30      # you can comment one or the other out to see what it does

plt.imshow(mysteryimage, cmap='gray')   
# the word cmap stands for 'colormap' 
# 'gray' specifies that you want to display a grayscale image
print('Type = ', type(mysteryimage))
print('Shape = ', np.shape(mysteryimage))

When you run the code, an 8x12 image should appear. Check that you understand each of these details. Write your answers to each question by editing this cell.
* *which set of indices goes with the rows and which with the colums?*
* *where is the origin (0,0) located?*
* *what is the data type of `mysteryimage`?*
* *what command can you use to find the dimensions of the image?*

**If you're interested:** take a look at how the `:` indexing works to create the pattern

### Loading and interpreting monochrome images

Here's how to read in an image into an array. Note that the code (as written) expects the file to be stored inside a folder named `pics` that is located in the same directory as this notebook. If that is not where you stored the image, you will need to edit that line.

In [ ]:
islands = io.imread(url) # <-- look what I put in the io.imread function there
plt.imshow(islands)
print('Type = ', type(islands))
print('Shape = ', np.shape(islands))

Notice that this image looks monochrome, but was (in fact) saved as a color image. Therefore, it is loaded as an $N \times M \times 3$ array. You can think of a color image as 3 arrays: one for the red channel, one for green channel, and one for the blue channel. Here, all three colors happen to be the same, so let's examine just one channel (I picked the red channel, but this was an arbitrary decision.)

In [ ]:
islands = islands[:,:,0]        # we're going to work with just the red (index=0) channel
dim = np.shape(islands)
plt.imshow(islands, cmap='gray')
plt.plot([0, dim[1]-1],[60,60])   # draw a line connecting the two endpoints of the row at index=60
plt.show()

In [ ]:
plt.plot(islands[60,:])
plt.xlabel('Pixel index (column)')
plt.ylabel('Image intensity value')
plt.show()

You can plot the data just along the blue row. Notice that there aren't any "units" associated with brightness. Most images are saved as integers (0 to 255) at each pixel.

**To do:** In the cell below, copy both codes above and then modify them so that you plot a single COLUMN instead of a single row.

In [ ]:
# put your code here

### Loading and interpreting color Images
The Hubble Ultra Deep Field image is a color image, and we will use that image to examine how colors are saved inside an image.

In [ ]:
deepfield = io.imread('https://raw.githubusercontent.com/lblogan14/CDSA2022_CompCourse/main/Day_10/HubbleDeepField.jpg')
plt.figure(figsize=(16,20))
plt.imshow(deepfield)
plt.show()

In [ ]:
crop = deepfield[900:1000,400:550,:] # crop a small portion of the large image
red = crop[:,:,0]                    # just the red channel: the #0 slice
blue = crop[:,:,2]                   # just the blue channel: the #2 slice

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,6))
ax1.imshow(red, cmap='gray')
ax1.set_title('Red Channel Only')
ax2.imshow(blue, cmap='gray')
ax2.set_title('Blue Channel Only')
ax3.imshow(crop)
ax3.set_title('Color Image')
plt.show()

Look at some of the colors of the astronomical objects. Based on the description at the top of the notebok, you should see redder-object and blue-objects. The redder objects in the Color Image show up primarily in the "Red Channel", while the bluer objects show up in the "Blue Channel"

**To do:**  In the cell below, copy the code above and then modify it ALSO show the green (index=1) channel, and also to crop the image to a different sub-region.

In [ ]:
# put your code here

## Exercise 2: Image segmentation

The process of locating discrete "objects" within an image is known as *image segmentation*, and there are a variety of algorithms used to complete this task. Here, we will use a very simple one: thresholding to find all regions above the mean brightness of the image.

In [ ]:
# Step 1: Find all regions above the mean
plt.imshow(islands, cmap='gray')
plt.title('original image')
plt.show()
threshold = islands > islands.mean()
# Ask yourself: what data type do you think the variable threshold is?
# Add a line of code here to check your answer



plt.imshow(threshold, cmap='gray')
plt.title('thresholded image')
plt.show()

In [ ]:
# Step 2: label each connected-component with a unique name
all_labels = measure.label(threshold)  
# Each region of the image "threshold" is saved into
# a new image named "all_labels" where the pixel has the 
# a value that is the label for that region

# more info about this funcion here:
# https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.label
# which also says a little bit about how it works
# you don't have to dig into the details: I just want you to know where they are

# we can see how this works by looking at the data inside the image
plt.imshow(all_labels,cmap='gray')
# for example:
row = 120
col = 170
plt.plot(col,row, 'rx')
plt.show()
print('x marks the region named', all_labels[row,col])
# To do: move the row and col values around to see the different
# name inside different regions

**To do:**  Inside the code above, move the row and col values around to see the different "name" inside different regions. The red x should move as well.

## Exercise 3: Region properties

We will use code from this library: https://scikit-image.org/docs/dev/api/skimage.measure.html

The "Notes" section under this heading is particuarly useful for learning about the various properties you can measure: https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprop

**To do:** Your task is to learn how these libraries work by using some examples given below. With your group, find out what each function is for. 

* **Option 1:** read the documentation for each of the functions we use below
* **Option 2:** see if you can figure out from the name of the function and its output what the function does, then you can check your answer against the documentation

When you have it figured out, add an informative label and units (if these exist) to each print statement below, in place of the useless text.

In [ ]:
# Step 3 Calculate image properties 

regions = measure.regionprops(all_labels)

print('Edit this line: ', len(regions))

print('Edit this line:', regions[52].centroid, '[pixels]')

rowscols = regions[5].coords
print('Edit this line:', rowscols[7,0], rowscols[7,1], islands[rowscols[7,0],rowscols[7,1]])

areas = [prop.area for prop in regions]
print('Edit this line:', areas[24])
count = 0
for i in range(len(regions)):
    if areas[i] > 10:
        count += 1
print('Edit this line:', count)

ecc = [prop.eccentricity for prop in regions]
print('Edit this line:', ecc[24])

## Exercise 4: Plot a histogram of areas

A histogram is a plot that lets you examine a population of measurements and see how the values are distributed. Characterizing this is one of the measurements the researchers studying this image need to do.

Below is an example of how to plot a histogram. Edit the code so that it plots the area data from Exercise 3 (rather than `randomdata`).

In [ ]:
randomdata = 5*np.random.randn(1000)    # this is a way to generate some random data
                                        # you'll want to comment out this line to do this exercise

    
#plot a histogram of your data (not the randomdata!)    
plt.hist(randomdata, bins=20)

#if you want more control over your bins, you can play with these 
#mybins = np.linspace(-20,20,21)
#plt.hist(randomdata, bins=mybins)

plt.xlabel('value [units?]')
plt.ylabel('# of occurances')
plt.show()

# OpenCV
What computer vision scientists actually use....
https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html

In [ ]:
import cv2
import urllib.request

In [ ]:
url_scene = 'https://raw.githubusercontent.com/lblogan14/CDSA2022_CompCourse/main/Day_10/input.jpg'
resp = urllib.request.urlopen(url_scene)
img = np.array(bytearray(resp.read()), dtype='uint8')
img = cv2.imdecode(img, cv2.IMREAD_COLOR)

In [ ]:
# something is not right...
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
plt.imshow(gray_img, cmap='gray')

## Image color spaces
* **RBG**: Red, Green, Blue. Every pixel value is represented as a tuple of three numbers corresponding to red, green, and blue. Each value ranges between 0 and 255
* **YUV**: Y refers to the luminance or intensity, U/V channels represent color information.
* **HSV**: Hue, Satruation, Value

## Convert color spaces
To see a list of all available flags of color spaces

In [ ]:
print([x for x in dir(cv2) if x.startswith('COLOR_')])

In [ ]:
# You can convert color space using cv2.cvtColor
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
plt.imshow(gray_img, cmap='gray')

## Split image channels
To convert color space to YUV

In [ ]:
yuv_img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
plt.imshow(yuv_img)

It may look weird at first.... Separate each channel to see the details:

In [ ]:
# alternative 1
y,u,v = cv2.split(yuv_img)
plt.figure(figsize=(15,12))
plt.subplot(311)
plt.imshow(y)
plt.title('Y Channel')
plt.subplot(312)
plt.imshow(u)
plt.title('U Channel')
plt.subplot(313)
plt.imshow(v)
plt.title('V Channel')

In [ ]:
# alternative 2 (faster)
plt.figure(figsize=(15,12))
plt.subplot(311)
plt.imshow(yuv_img[:,:,0], label='Y Channel')
plt.subplot(312)
plt.imshow(yuv_img[:,:,1], label='U Channel')
plt.subplot(313)
plt.imshow(yuv_img[:,:,2], label='V Channel')

## Merge image channels
read an image, split it into separate channels, and merge them to see how different effects can be obtained out of different combinations

In [ ]:
r,g,b = cv2.split(img)
gbr_img = cv2.merge((g,b,r))
rbr_img = cv2.merge((r,b,r))
rgb_img = cv2.merge((r,g,b))

plt.figure(figsize=(15,12))
plt.subplot(311)
plt.imshow(rgb_img)
plt.title('Original Image')
plt.subplot(312)
plt.imshow(gbr_img)
plt.title('GBR Image')
plt.subplot(313)
plt.imshow(rbr_img)
plt.title('RBR Image')

## Image translation

shifting an image

In [ ]:
img = rgb_img
num_rows, num_cols = img.shape[:2]
translation_matrix = np.float32([ [1,0,70], [0,1,110]])
img_translation = cv2.warpAffine(img, translation_matrix, (num_cols, num_rows), cv2.INTER_LINEAR)
plt.imshow(img_translation)
plt.title('Translation')

Translation means to shift the image by adding/substracting the *x* and *y* coordinates. To do this, need to create a transformation matrix defined as follows,
$$T = \begin{bmatrix} 1 & 0 & t_x \\ 0 & 1& t_y \end{bmatrix}$$
The $t_x$ and $t_y$ values are the *x* and *y* translation values. The image will be moved by *x* units to the right and by *y* units downwards. Then we use the `warpAffine` function to translate the image. The third argument in `warpAffine` refers to the number of rows and columns in the resulting image. It passes `InterpolaitonFlags` which defines combination of interpolation methods. In this case, it is `INTER_LINEAR` method.

Snce the number of rows and columns is the same as the original image, the resultant
image is going to get cropped. The reason for this is that the display space is not enough in the
output when we applied the translation matrix. To avoid cropping,

In [ ]:
img_translation = cv2.warpAffine(img, translation_matrix, (num_cols+70, num_rows+110))
plt.imshow(img_translation)
plt.title('Translation without chopping')

To move the image to the middle of a bigger image frame,

In [ ]:
translation_matrix = np.float32([[1,0,70], [0,1,110]])
img_translation = cv2.warpAffine(img, 
                                 translation_matrix, 
                                 (num_cols+70, num_rows+110)
                                )
translation_matrix = np.float32([[1,0,-30], [0,1,-50]])
img_translation = cv2.warpAffine(img_translation, 
                                 translation_matrix, 
                                 (num_cols+70+30, num_rows+110+50))
plt.imshow(img_translation)

`borderMode` and `borderValue` allow you to fill up the empty borders of the translation with a pixel extrapolation method,

In [ ]:
translation_matrix = np.float32([[1,0,70], [0,1,110]])
img_translation = cv2.warpAffine(img, 
                                 translation_matrix, 
                                 (num_cols+70, num_rows+110),
                                 cv2.INTER_LINEAR,
                                 cv2.BORDER_WRAP, # <- borderMode
                                 1) # <- borderValue
plt.imshow(img_translation)

## Image rotation

In [ ]:
rotation_matrix = cv2.getRotationMatrix2D((num_cols/2, num_rows/2),
                                          30,
                                          0.7)
img_rotation = cv2.warpAffine(img, rotation_matrix, (num_cols, num_rows))
plt.imshow(img_rotation)

`getRotationMatrix2D` specify the center point around which the image would be rotated as the first argument, then the angl of rotation in degrees, and a scaling factor for the image at the end. `30` as the second argument is to rotated the image by 30 degrees and `0.7` is to shrink the image by 30%

In [ ]:
rotation_matrix

## Image scaling

Resize an image

In [ ]:
img_scaled = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
plt.imshow(img_scaled)
plt.title('Scaling - Linear interpolation')

In [ ]:
img_scaled = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
plt.imshow(img_scaled)
plt.title('Scaling - Cubic Interpolation')

The `fx` and `fy` are the scaling factors. In this case, the image will be enlarged by a factor of 1.5

In [ ]:
img_scaled = cv2.resize(img, (450, 400), interpolation=cv2.INTER_AREA)
plt.imshow(img_scaled)
plt.title('Scaling - Skewed Size')

If the `None` arg is replaced by a specific size, in this case, `(450, 400)`, the `resize` function will use that size to skew the image and resize it to that size.

## Affine transformations
**Euclidean transformations** are a type of geometric transformation that preserve length and
angle measures.
 If we take a geometric shape and apply Euclidean transformation to it, the
shape will remain unchanged. It might look rotated, shifted, and so on, but the basic
structure will not change.

**Affine transformations** are generalizations of Euclidean transformations.  Under the realm of affine transformations, lines will remain lines, but squares might become rectangles or parallelograms. Basically, affine
transformations don't preserve lengths and angles.

In order to build a general affine transformation matrix, we need to define the control
points. Once we have these control points, we need to decide where we want them to be
mapped.

 In this particular situation, all we need are three points in the source image, and
three points in the output image. If we want to convert an image into a parallelogram-like image,

In [ ]:
src_points = np.float32([[0,0], 
                         [num_cols-1,0], 
                         [0,num_rows-1]])
dst_points = np.float32([[0,0], 
                         [int(0.6*(num_cols-1)),0], 
                         [int(0.4*(num_cols-1)),num_rows-1]])

affine_matrix = cv2.getAffineTransform(src_points, dst_points)
img_output = cv2.warpAffine(img, affine_matrix, (num_cols, num_rows))
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.imshow(img)
plt.title('Input')
plt.subplot(212)
plt.imshow(img_output)
plt.title('Output')

## Projective transformations
Any two images on a given plane are related by a homography. Once the camera rotation and translation have been extracted from an estimated
homography matrix, this information may be used for navigation, or to insert models of 3D
objects into an image or video

In [ ]:
src_points = np.float32([[0,0], 
                         [num_cols-1,0], 
                         [0,num_rows-1], 
                         [num_cols-1,num_rows-1]])
dst_points = np.float32([[0,0], [num_cols-1,0], 
                         [int(0.33*num_cols),num_rows-1],
                         [int(0.66*num_cols),num_rows-1]])

projective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
img_output = cv2.warpPerspective(img, projective_matrix, (num_cols, num_rows))
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.imshow(img)
plt.title('Input')
plt.subplot(212)
plt.imshow(img_output)
plt.title('Output')

Choose four control points in the source image and map them to the destination
image. Parallel lines will not remain parallel lines after the transformation. The `getPerspectiveTransform()` function is used to get the transformation matrix.

In [ ]:
src_points = np.float32([[0,0], 
                         [0,num_rows-1], 
                         [num_cols/2,0],
                         [num_cols/2,num_rows-1]])
dst_points = np.float32([[0,100], 
                         [0,num_rows-101],
                         [num_cols/2,0],
                         [num_cols/2,num_rows-1]])
projective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
img_output = cv2.warpPerspective(img, projective_matrix, (num_cols, num_rows))
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.imshow(img)
plt.title('Input')
plt.subplot(212)
plt.imshow(img_output)
plt.title('Output')

## Image warping
Create a custom mapping

In [ ]:
img.shape

In [ ]:
gray_img.shape

In [ ]:
rows, cols = gray_img.shape

In [ ]:
# vertical wave
img_output = np.zeros(gray_img.shape, dtype=gray_img.dtype)

for i in range(rows):
  for j in range(cols):
    offset_x = int(25.0 * np.sin(2*3.14*i/180))
    offset_y = 0
    if j+offset_x < rows:
      img_output[i,j] = gray_img[i,(j+offset_x)%cols]
    else:
      img_output[i,j] = 0
      
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.imshow(gray_img, cmap='gray')
plt.title('Input')
plt.subplot(212)
plt.imshow(img_output, cmap='gray')
plt.title('Output')

In [ ]:
# horizontal wave
img_output = np.zeros(gray_img.shape, dtype=gray_img.dtype)

for i in range(rows):
  for j in range(cols):
    offset_x = 0
    offset_y = int(16.0*np.sin(2*3.14*j/150))
    if i+offset_y < cols:
      img_output[i,j] = gray_img[(i+offset_y)%rows, j]
    else:
      img_output[i,j] = 0
      
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.imshow(gray_img, cmap='gray')
plt.title('Input')
plt.subplot(212)
plt.imshow(img_output, cmap='gray')
plt.title('Output')

In [ ]:
# both horizontal and vertical waves
img_output = np.zeros(gray_img.shape, dtype=gray_img.dtype) 
 
for i in range(rows): 
    for j in range(cols): 
        offset_x = int(20.0 * np.sin(2 * 3.14 * i / 150)) 
        offset_y = int(20.0 * np.cos(2 * 3.14 * j / 150)) 
        if i+offset_y < rows and j+offset_x < cols: 
            img_output[i,j] = gray_img[(i+offset_y)%rows,(j+offset_x)%cols] 
        else: 
            img_output[i,j] = 0 
            
plt.figure(figsize=(15,12))
plt.subplot(211)
plt.imshow(gray_img, cmap='gray')
plt.title('Input')
plt.subplot(212)
plt.imshow(img_output, cmap='gray')
plt.title('Output')

## Blurring
known as **low pass filter**
A low pass filter is a filter that allows low frequencies, and blocks higher
frequencies. The frequency in an image refers to the rate of change of pixel values. So the sharp edges would be high-frequency content because the pixel values change rapidly in that region, and the plain areas would be low-frequency
content. Going by this definition, a low pass filter would try to smooth the edges.

A simple way to build a low pass filter is by uniformly averaging the values in the
neighborhood of a pixel.

In [ ]:
kernel_identity = np.array([[0,0,0], [0,1,0], [0,0,0]])
kernel_4x4 = np.ones((4,4), np.float32) / 16.0
kernel_6x6 = np.ones((6,6), np.float32) / 36.0

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(img)
plt.title('Original')

In [ ]:
output = cv2.filter2D(img, -1, kernel_identity)
# value -1 is to maintain source image depth
plt.figure(figsize=(10,8))
plt.imshow(output)
plt.title('Identity Filter')

In [ ]:
output = cv2.filter2D(img, -1, kernel_4x4)
plt.figure(figsize=(10,8))
plt.imshow(output)
plt.title('4x4 Filter')

In [ ]:
output = cv2.filter2D(img, -1, kernel_6x6)
plt.figure(figsize=(10,8))
plt.imshow(output)
plt.title('6x6 Filter')

## Size of the kernel versus blurriness
In the previous examples, the `filter2D` function is applied to the input image. As the kernel size increases, the images get blurrier.

Another way to do this is to apply the `blur` function if you don't want to generate the kernels by yourself.

In [ ]:
output = cv2.blur(img, (6,6))
plt.figure(figsize=(10,8))
plt.imshow(output)
plt.title('6x6 Filter')

#Motion blur
When we apply the motion blurring effect, it will look like you captured the picture while
moving in a particular direction

In [ ]:
size = 15

# generate kernel
kernel_motion_blur = np.zeros((size, size))
kernel_motion_blur[int((size-1)/2), :] = np.ones(size)
kernel_motion_blur = kernel_motion_blur / size

# apply kernel to input image
output = cv2.filter2D(img, -1, kernel_motion_blur)

plt.figure(figsize=(10,8))
plt.imshow(img)
plt.title('Original')
plt.figure(figsize=(10,8))
plt.imshow(output)
plt.title('Motion Blur')

The motion blur kernel averages the pixel values in a particular direction. It's like a directional low pass filter. 

For example, a 3x3 horizontal motion-blurring kernel will look like this,
$$M=\begin{bmatrix} 0 & 0 & 0 \\ 1 & 1 & 1 \\ 0 & 0 & 0 \end{bmatrix}$$
This will blur the image in a horizontal direction.

The example shown above applies a 15x15 kernel.

#Sharpening
Applying the sharpening filter will sharpen the edges in the image. This filter is very useful
when we want to enhance the edges of an image that's not crisp enough

In [ ]:
# generate kernels
kernel_sharpen_1 = np.array([[-1,-1,-1], 
                             [-1,9,-1], 
                             [-1,-1,-1]])
kernel_sharpen_2 = np.array([[1,1,1],
                             [1,-7,1],
                             [1,1,1]])
kernel_sharpen_3 = np.array([[-1,-1,-1,-1,-1],
                             [-1,2,2,2,-1], 
                             [-1,2,8,2,-1], 
                             [-1,2,2,2,-1], 
                             [-1,-1,-1,-1,-1]]) / 8.0 

# apply kernels to input image
output_1 = cv2.filter2D(img, -1, kernel_sharpen_1)
output_2 = cv2.filter2D(img, -1, kernel_sharpen_2)
output_3 = cv2.filter2D(img, -1, kernel_sharpen_3)

To just sharpen the image, we can apply a kernel like this,
$$M = \begin{bmatrix} -1 & -1 & -1 \\ -1 & 9 & -1 \\ -1 & -1 & -1 \end{bmatrix}$$
and the result is shown below,

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(output_1)
plt.title('Sharpening')

To just excessively sharpen the image, we can apply a kernel like this,
$$M = \begin{bmatrix} 1 & 1 & 1 \\ 1 & -7 & 1 \\ 1 & 1 & 1 \end{bmatrix}$$
and the result is shown below,

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(output_2)
plt.title('Excessive Sharpening')

If we want our images to look more natural, we would use an edge enhancement filter. The
underlying concept remains the same, but we use an approximate Gaussian kernel to build
this filter. It will help us smooth the image when we enhance the edges, thus making the
image look more natural.

To achievethis, we can apply a kernel like this,
$$M = \begin{bmatrix} -1 & -1 & -1 & -1 & -1\\ -1 & 2 & 2 & 2 & -1 \\ -1 & 2 & 8 & 2 & -1 \\  -1 & 2 & 2 & 2 & -1 \\ -1 & -1 & -1 & -1 & -1\end{bmatrix}$$
and the result is shown below,

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(output_3)
plt.title('Edge Enhancement')

#Embossing
An embossing filter will take an image and convert it to an embossed image. Each pixel will be replaced with a shadow or a highlight.

In [ ]:
# generate kernels
kernel_emboss_1 = np.array([[0,-1,-1],
                            [1,0,-1],
                            [1,1,0]])
kernel_emboss_2 = np.array([[-1,-1,0],
                            [-1,0,1],
                            [0,1,1]])
kernel_emboss_3 = np.array([[1,0,0],
                            [0,0,0],
                            [0,0,-1]])

# convert input image to grayscale
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# apply kernels to grayscale image and add offset to produce the shadow
output_1 = cv2.filter2D(gray_img, -1, kernel_emboss_1) + 128
output_2 = cv2.filter2D(gray_img, -1, kernel_emboss_2) + 128
output_3 = cv2.filter2D(gray_img, -1, kernel_emboss_3) + 128

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(img)
plt.title('Original')
plt.figure(figsize=(10,8))
plt.imshow(output_1, cmap='gray')
plt.title('Embossing - South West')
plt.figure(figsize=(10,8))
plt.imshow(output_2, cmap='gray')
plt.title('Embossing - South East')
plt.figure(figsize=(10,8))
plt.imshow(output_3, cmap='gray')
plt.title('Embossing - North West')

The embossing effect is
achieved by offsetting all the pixel values in the image by `128`. This operation adds the
highlight/shadow effect to the picture.

#Edge detection
The process of edge detection involves detecting sharp edges in the image, and producing a
binary image as the output. Edge detection can be thought of as a high pass filtering operation.  A
high pass filter allows high-frequency content to pass through and blocks the low-frequency
content. As we discussed earlier, edges are high-frequency content. In edge detection, we
want to retain these edges and discard everything else. 

`Sobel` filter is composed of,
$$S_x=\begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix} \quad\quad\quad 
S_y =\begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 &0 \\ 1 & 2 & 1 \end{bmatrix}$$
$S_x$ detects horizontal edges and $S_y$ detects vertical edges

In [ ]:
url_geo = 'https://raw.githubusercontent.com/lblogan14/CDSA2022_CompCourse/main/Day_10/geometrics_input.png'
resp = urllib.request.urlopen(url_geo)
img_geo = np.array(bytearray(resp.read()), dtype='uint8')
img_geo = cv2.imdecode(img_geo, cv2.IMREAD_GRAYSCALE)

In [ ]:
# it is used depth of cv2.CV_64F
sobel_horizontal = cv2.Sobel(img_geo, cv2.CV_64F, 1, 0, ksize=5)
# kernel size can be: 1, 3, 5, 7
sobel_vertical = cv2.Sobel(img_geo, cv2.CV_64F, 0, 1, ksize=5)

In the case of 8-bit input images, it will result in truncated derivatives, so depth
value `cv2.CV_16U` can be used instead. In case edges are not that well-defined the value
of kernel can be adjusted, minor to obtain thinner edges and major for the opposite purpose. 

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(img_geo, cmap='gray')
plt.title('Original')
plt.figure(figsize=(10,8))
plt.imshow(sobel_horizontal, cmap='gray')
plt.title('Sobel horizontal')
plt.figure(figsize=(10,8))
plt.imshow(sobel_vertical, cmap='gray')
plt.title('Sobel vertical')
laplacian = cv2.Laplacian(img_geo, cv2.CV_64F)
plt.figure(figsize=(10,8))
plt.imshow(laplacian, cmap='gray')
plt.title('Laplacian')

`Laplacian` does not work well given too much noise in an image. In this case, try `Canny` edge detector,

In [ ]:
url_train = 'https://raw.githubusercontent.com/lblogan14/CDSA2022_CompCourse/main/Day_10/train_input.png'
resp = urllib.request.urlopen(url_train)
img_train = np.array(bytearray(resp.read()), dtype='uint8')
img_train = cv2.imdecode(img_train, cv2.IMREAD_GRAYSCALE)

In [ ]:
laplacian = cv2.Laplacian(img_train, cv2.CV_64F)
canny = cv2.Canny(img_train, 50, 240)

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(img_train, cmap='gray')
plt.title('Original')
plt.figure(figsize=(10,8))
plt.imshow(laplacian, cmap='gray')
plt.title('Laplacian')
plt.figure(figsize=(10,8))
plt.imshow(canny, cmap='gray')
plt.title('Canny')

The Laplacian kernel gives rise to a noisy output, the edges are not specified much clearly shown in the image.

The `Canny` edge detector is much better. 

`Canny` takes two numbers as arguments to indicate the thresholds. The second argument is the **low threshold
value**, and the third argument is the **high threshold value**. If the gradient value is
beyond the high threshold value, it is marked as a strong edge. The `Canny` edge detector
starts tracking the edge from this point and continues the process until the gradient value
falls below the low threshold value. As you increase these thresholds, the weaker edges will
be ignored. The output image will be cleaner and sparser. 

## Creating a vignette filter

In [ ]:
url_flr = 'https://raw.githubusercontent.com/lblogan14/CDSA2022_CompCourse/main/Day_10/flower_input.png'
resp = urllib.request.urlopen(url_flr)
img_flr = np.array(bytearray(resp.read()), dtype='uint8')
img_flr = cv2.imdecode(img_flr, cv2.IMREAD_COLOR)
img_flr = cv2.cvtColor(img_flr, cv2.COLOR_BGR2RGB)
rows, cols = img_flr.shape[:2]

In [ ]:
# generate vignette mask using Gaussian kernels
kernel_x = cv2.getGaussianKernel(cols, 200)
kernel_y = cv2.getGaussianKernel(rows, 200)
kernel = kernel_y * kernel_x.T
mask = 255 * kernel / np.linalg.norm(kernel)
output = np.copy(img_flr)

# apply mask to each channel in the input image
for i in range(3):
  output[:,:,i] = output[:,:,i] * mask

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(img_flr)
plt.title('Original')
plt.figure(figsize=(10,8))
plt.imshow(output)
plt.title('Vignette')

The vignette filter basically focuses the brightness on a particular part of the image and the
other parts look faded. In order to achieve this, we need to filter out each channel in the
image using a Gaussian kernel, by `getGaussianKernel` function.

The second parameter of `getGaussianKernel` is the standard deviation of the Gaussian and controls the radius of the bright central region.

Once the 2D kernel is built, a mask is built by normalizing this kernel and scaling it up: \\
`mask = 255 * kernel/np.linalg.norm(kernel)` \\
This is an important step because if you don't scale it up, the image will look black. This
happens because all the pixel values will be close to zero after you superimpose the mask
on the input image.

To focus on a different region in the image,

In [ ]:
# generating vignette mask using Gaussian kernels 
kernel_x = cv2.getGaussianKernel(int(1.5*cols),200) 
kernel_y = cv2.getGaussianKernel(int(1.5*rows),200) 
kernel = kernel_y * kernel_x.T 
mask = 255 * kernel / np.linalg.norm(kernel) 
mask = mask[int(0.5*rows):, int(0.5*cols):] 
output = np.copy(img_flr) 
 
# applying the mask to each channel in the input image 
for i in range(3): 
  output[:,:,i] = output[:,:,i] * mask 

All we need to do is build a bigger Gaussian kernel, and make sure that the peak coincides
with the region of interest.

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(img_flr)
plt.title('Original')
plt.figure(figsize=(10,8))
plt.imshow(output)
plt.title('Vignette')

## Enhance the contrast in an image
The pixel values tend to concentrate near zero when we capture the images in a low-light condition. When this happens, a lot of details in the image are not clearly visible to the human eye. Use the **histogram equalization** to enhance the contrast to capture the details.

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(gray_img, cmap='gray')
plt.title('Input')

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(gray_img.flatten())
plt.title('Histogram of Input')

In [ ]:
# equalize the histogram of input image
histeq = cv2.equalizeHist(gray_img)

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(histeq.flatten())
plt.title('Histogram of Hist-Equalized Input')

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(gray_img, cmap='gray')
plt.title('Original Input')

plt.figure(figsize=(10,8))
plt.imshow(histeq, cmap='gray')
plt.title('Histogram equalized')

The  histogram equalization is that it's a nonlinear process. So, we cannot just separate out the three channels in an RGB image,
equalize the histogram separately, and combine them later to form the output image.

In order to handle the histogram equalization of color images, we need to convert it to a
color space, where intensity is separated from the color information. YUV would be a good choice because the YUV model defines a color space in terms of one **Luminance (Y)** and two **Chrominance (UV)** components. Once we convert it to YUV, we
just need to equalize the Y-channel and combine it with the other two channels to get the
output image.